# Mistral AI Models Test Notebook (Text · Reasoning · Multimodal · Embeddings)


This notebook helps you **try different Mistral AI models by category** (Text, Reasoning, Multimodal, and Embeddings), with **pricing snapshots** and ready-to-run test cells using the latest Python SDK style.

> **Sources for pricing & models (check for updates):**
> - Mistral AI API Pricing (official): https://mistral.ai/pricing/
> - Platform documentation: https://docs.mistral.ai/
> - API reference: https://docs.mistral.ai/api/

> ⚠️ **Always verify prices** on the official pricing page before production use.

## 0) Setup

1. Install the latest Mistral SDK:
   ```bash
   pip install --upgrade mistralai
   ```

2. Export your API key (or use `.env`):
   ```bash
   export MISTRAL_API_KEY="your_api_key"
   ```

3. Run cells below. Networking must be enabled in your environment to call the API.

In [ ]:
# Optional: !pip install python-dotenv mistralai pandas matplotlib ipywidgets
import os
import json
import time
import pathlib
import math
import base64
from typing import Dict, Any, Optional, List, Union

import pandas as pd
import matplotlib.pyplot as plt

try:
    # Import dotenv for loading .env file if available
    from dotenv import load_dotenv
    load_dotenv()
except ImportError:
    print("dotenv not available. Install with `pip install python-dotenv`")

try:
    from mistralai.client import MistralClient
    from mistralai.models.chat_completion import ChatMessage
    from mistralai.exceptions import MistralAPIException
    
    # Initialize the Mistral client
    api_key = os.environ.get("MISTRAL_API_KEY")
    if not api_key:
        print("⚠️ MISTRAL_API_KEY environment variable not found. Please set it before proceeding.")
    else:
        client = MistralClient(api_key=api_key)
        print("✅ MistralClient initialized successfully.")
except ImportError:
    print("Mistral SDK not available. Install with `pip install mistralai`")
except Exception as e:
    print(f"Error initializing MistralClient: {e}")

## 1) Pricing snapshot (as of this notebook's creation)

> Verify current prices at https://mistral.ai/pricing/ before use.

We include commonly used models across categories. All Mistral AI models bill **per token** (input and output tokens).

In [ ]:
import pandas as pd

# Updated pricing snapshot — verify on https://mistral.ai/pricing/ before production.
# Rates are per 1M tokens.

PRICING = [
    # ---------- TEXT MODELS ----------
    # Mistral Large Family
    {"model": "mistral-large-latest", "category": "text", "input": 8.00, "output": 24.00, "context_length": 32768, "unit": "USD / 1M tokens", "notes": "Most capable model for complex tasks"},
    {"model": "mistral-large-2407", "category": "text", "input": 8.00, "output": 24.00, "context_length": 32768, "unit": "USD / 1M tokens", "notes": "July 2024 model"},
    
    # Mistral Medium Family
    {"model": "mistral-medium-latest", "category": "text", "input": 2.70, "output": 8.10, "context_length": 32768, "unit": "USD / 1M tokens", "notes": "Balance between capabilities and cost"},
    {"model": "mistral-medium-2407", "category": "text", "input": 2.70, "output": 8.10, "context_length": 32768, "unit": "USD / 1M tokens", "notes": "July 2024 model"},
    
    # Mistral Small Family
    {"model": "mistral-small-latest", "category": "text", "input": 0.70, "output": 2.10, "context_length": 32768, "unit": "USD / 1M tokens", "notes": "Cost-effective for general tasks"},
    {"model": "mistral-small-2407", "category": "text", "input": 0.70, "output": 2.10, "context_length": 32768, "unit": "USD / 1M tokens", "notes": "July 2024 model"},
    
    # Mistral Tiny Family
    {"model": "mistral-tiny-2407", "category": "text", "input": 0.14, "output": 0.42, "context_length": 32768, "unit": "USD / 1M tokens", "notes": "Most efficient model for simple tasks"},
    
    # Open Models
    {"model": "open-mistral-7b", "category": "text", "input": 0.25, "output": 0.25, "context_length": 8192, "unit": "USD / 1M tokens", "notes": "Open-weight 7B model"},
    {"model": "open-mixtral-8x7b", "category": "text", "input": 0.70, "output": 0.70, "context_length": 32768, "unit": "USD / 1M tokens", "notes": "Open-weight MoE model"},
    
    # ---------- MULTIMODAL MODELS ----------
    {"model": "mistral-large-vision-2407", "category": "multimodal", "input": 8.00, "output": 24.00, "context_length": 32768, "unit": "USD / 1M tokens", "notes": "Vision model with image understanding"},
    
    # ---------- EMBEDDING MODELS ----------
    {"model": "mistral-embed", "category": "embedding", "input": 0.10, "output": None, "context_length": 8192, "unit": "USD / 1M tokens", "notes": "Text embedding model (1024D)"},
    
    # ---------- REASONING MODELS ----------
    # NOTE: Reasoning capability is integrated into the main models, particularly large & medium
    {"model": "mistral-large-latest", "category": "reasoning", "input": 8.00, "output": 24.00, "context_length": 32768, "unit": "USD / 1M tokens", "notes": "Most capable for complex reasoning"},
    {"model": "mistral-medium-latest", "category": "reasoning", "input": 2.70, "output": 8.10, "context_length": 32768, "unit": "USD / 1M tokens", "notes": "Good reasoning with balanced cost"}
]

df_pricing = pd.DataFrame(PRICING)
df_pricing

## 2) Recommendations & light rankings

Below is a simple, **opinionated** ranking by category. Adjust for your workload.

In [ ]:
import pandas as pd

# Heuristic ranking (lower score = better considering price/perf for the category)
RANKING = {
    "text": [
        {"model": "mistral-large-latest", "score": 1, "why": "Best overall performance for challenging tasks"},
        {"model": "mistral-medium-latest", "score": 2, "why": "Great balance of capability and cost efficiency"},
        {"model": "mistral-small-latest", "score": 3, "why": "Good performance for most general tasks at lower cost"}
    ],
    "multimodal": [
        {"model": "mistral-large-vision-2407", "score": 1, "why": "High-performance vision capabilities"}
    ],
    "reasoning": [
        {"model": "mistral-large-latest", "score": 1, "why": "Best for complex reasoning and multi-step problem solving"},
        {"model": "mistral-medium-latest", "score": 2, "why": "Good reasoning capabilities with better cost efficiency"},
        {"model": "mistral-small-latest", "score": 3, "why": "Adequate for simpler reasoning tasks at an affordable price"}
    ],
    "embedding": [
        {"model": "mistral-embed", "score": 1, "why": "High-quality embeddings for semantic search and RAG"}
    ]
}

def ranking_table(category: str):
    rows = RANKING.get(category, [])
    return pd.DataFrame(rows)

display(ranking_table("text"))
display(ranking_table("multimodal"))
display(ranking_table("reasoning"))
display(ranking_table("embedding"))

## 3) Test harness

Utilities to run quick tests against selected models using the **Chat API**.

> Make sure `MISTRAL_API_KEY` is set.

In [ ]:
from typing import Dict, Any, List, Optional, Tuple
import time
from mistralai.models.chat_completion import ChatMessage

def run_chat_with_model(model: str, messages: List[ChatMessage], temperature: float = 0.7, max_tokens: Optional[int] = None) -> Dict[str, Any]:
    """Sends a list of chat messages and returns the model's response and usage."""
    assert client is not None, "MistralClient not initialized. Set MISTRAL_API_KEY and initialize client."
    
    try:
        response = client.chat(
            model=model,
            messages=messages,
            temperature=temperature,
            max_tokens=max_tokens
        )
        
        content = response.choices[0].message.content
        usage = response.usage
        
        return {
            "success": True,
            "content": content,
            "usage": usage,
            "model": model,
            "finish_reason": response.choices[0].finish_reason
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

def run_text_prompt(model: str, prompt: str, system_prompt: str = None, temperature: float = 0.7, max_tokens: Optional[int] = None) -> Dict[str, Any]:
    """Sends a simple text prompt and returns the text output and usage."""
    messages = []
    
    if system_prompt:
        messages.append(ChatMessage(role="system", content=system_prompt))
        
    messages.append(ChatMessage(role="user", content=prompt))
    
    return run_chat_with_model(model, messages, temperature, max_tokens)

def estimate_cost(model: str, usage: Dict[str, Any], pricing_df) -> Tuple[float, float, float]:
    """Estimates input, output, and total costs given usage metrics and our pricing table."""
    if not usage:
        return 0.0, 0.0, 0.0
    
    prompt_tokens = usage.get("prompt_tokens", 0)
    completion_tokens = usage.get("completion_tokens", 0)

    # Find the model in pricing dataframe
    model_row = pricing_df[pricing_df["model"] == model]
    if model_row.empty:
        # Try to find by prefix match
        for idx, row in pricing_df.iterrows():
            if model.startswith(row["model"].split("-latest")[0]):
                model_row = pricing_df.iloc[[idx]]
                break
    
    if model_row.empty:
        return 0.0, 0.0, 0.0

    input_rate = model_row.iloc[0]["input"] or 0.0
    output_rate = model_row.iloc[0]["output"] or 0.0
    
    input_cost = (prompt_tokens / 1_000_000.0) * input_rate
    output_cost = (completion_tokens / 1_000_000.0) * output_rate
    total_cost = input_cost + output_cost
    
    return input_cost, output_cost, total_cost

### 4A) Text models tests

Use different models in the Mistral AI family to compare output quality, performance, and cost.

In [ ]:
# Test a basic prompt with different text models
prompt = "Write a 4-sentence summary of the benefits of Infrastructure as Code (IaC) in Kubernetes environments."

text_models = ["mistral-tiny-2407", "mistral-small-latest", "mistral-medium-latest", "mistral-large-latest"]
results = {}

for model in text_models:
    try:
        print(f"\n📝 Testing model: {model}")
        print("-" * 50)
        
        result = run_text_prompt(model=model, prompt=prompt)
        
        if result["success"]:
            print(result["content"])
            
            # Calculate cost
            input_cost, output_cost, total_cost = estimate_cost(model, result["usage"], df_pricing)
            
            print(f"\n📊 Usage:")
            print(f"  Prompt tokens: {result['usage'].prompt_tokens}")
            print(f"  Completion tokens: {result['usage'].completion_tokens}")
            print(f"  Total tokens: {result['usage'].prompt_tokens + result['usage'].completion_tokens}")
            print(f"  Estimated cost: ${total_cost:.6f} (${input_cost:.6f} input + ${output_cost:.6f} output)")
            
            results[model] = {
                "content": result["content"],
                "usage": result["usage"],
                "cost": total_cost
            }
        else:
            print(f"❌ Error: {result['error']}")
            
    except Exception as e:
        print(f"❌ Error with model {model}: {e}")

### 4B) Complex Reasoning Tests

Test the reasoning capabilities of Mistral AI models with more complex tasks that require multi-step thinking.

In [ ]:
# Test reasoning capabilities with a math problem
reasoning_prompt = "Solve this: If f(x)=2x^2-3x+5, compute f(7). Show steps briefly, then give the final answer on a new line prefixed with 'Answer:'."

# We'll test with medium and large models which excel at reasoning tasks
reasoning_models = ["mistral-small-latest", "mistral-medium-latest", "mistral-large-latest"]

for model in reasoning_models:
    try:
        print(f"\n🧠 Testing reasoning with: {model}")
        print("-" * 50)
        
        result = run_text_prompt(
            model=model,
            prompt=reasoning_prompt,
            temperature=0.3,  # Lower temperature for more precise reasoning
            max_tokens=500
        )
        
        if result["success"]:
            print(result["content"])
            
            # Calculate cost
            input_cost, output_cost, total_cost = estimate_cost(model, result["usage"], df_pricing)
            
            print(f"\n📊 Usage:")
            print(f"  Prompt tokens: {result['usage'].prompt_tokens}")
            print(f"  Completion tokens: {result['usage'].completion_tokens}")
            print(f"  Estimated cost: ${total_cost:.6f}")
        else:
            print(f"❌ Error: {result['error']}")
            
    except Exception as e:
        print(f"❌ Error with model {model}: {e}")
        
# Now let's test a more complex reasoning problem - a logical puzzle
complex_reasoning_prompt = """
Solve this logical puzzle step by step:

Five friends (Alex, Blake, Casey, Dana, and Elliot) are sitting in a row at a movie theater. 
We know the following:
- Alex is sitting to the left of Blake
- Casey is sitting next to Dana
- Elliot is sitting at one of the ends
- Blake is sitting next to Elliot
- Alex and Dana are not sitting next to each other

What is the seating arrangement from left to right?
"""

print("\n🧩 Complex Logical Reasoning Test")
print("=" * 60)

# Testing with the most capable model for complex reasoning
try:
    result = run_text_prompt(
        model="mistral-large-latest",
        prompt=complex_reasoning_prompt,
        temperature=0.3,
        max_tokens=1000
    )
    
    if result["success"]:
        print(result["content"])
        
        # Calculate cost
        input_cost, output_cost, total_cost = estimate_cost("mistral-large-latest", result["usage"], df_pricing)
        print(f"\n📊 Usage:")
        print(f"  Prompt tokens: {result['usage'].prompt_tokens}")
        print(f"  Completion tokens: {result['usage'].completion_tokens}")
        print(f"  Estimated cost: ${total_cost:.6f}")
    else:
        print(f"❌ Error: {result['error']}")
        
except Exception as e:
    print(f"❌ Error with complex reasoning: {e}")

### 4C) Multimodal (Vision) Tests

Test the vision capabilities of Mistral AI models with image understanding tasks.

In [ ]:
def encode_image_to_base64(image_path):
    """Encodes an image file to base64 format."""
    import base64
    import os
    
    if not os.path.exists(image_path):
        raise FileNotFoundError(f"Image file not found: {image_path}")
    
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def run_vision_query(image_path, prompt, model="mistral-large-vision-2407", temperature=0.5):
    """Send a vision query with an image and a prompt."""
    try:
        # Encode image
        base64_image = encode_image_to_base64(image_path)
        
        # Create a multimodal message
        messages = [
            ChatMessage(
                role="user",
                content=[
                    {"type": "text", "text": prompt},
                    {"type": "image", "image": {"data": base64_image, "format": "jpeg"}}
                ]
            )
        ]
        
        # Call the API
        response = client.chat(
            model=model,
            messages=messages,
            temperature=temperature
        )
        
        return {
            "success": True,
            "content": response.choices[0].message.content,
            "usage": response.usage
        }
    
    except FileNotFoundError as e:
        return {"success": False, "error": str(e)}
    except Exception as e:
        return {"success": False, "error": str(e)}

# Test vision capabilities if an image is available
# First check if a sample image exists
import os

sample_image_path = "sample_image.jpg"  # Replace with your image path

if os.path.exists(sample_image_path):
    print("🖼️ Testing vision capabilities with sample image")
    print("-" * 50)
    
    vision_prompts = [
        "Describe what you see in this image in detail.",
        "What objects can you identify in this image?",
        "What is the main subject of this image? Analyze its composition."
    ]
    
    for prompt in vision_prompts:
        try:
            print(f"\n📝 Vision prompt: {prompt}")
            result = run_vision_query(sample_image_path, prompt)
            
            if result["success"]:
                print(f"\n💬 Response:")
                print(result["content"])
                
                # Calculate cost
                input_cost, output_cost, total_cost = estimate_cost("mistral-large-vision-2407", result["usage"], df_pricing)
                print(f"\n📊 Usage:")
                print(f"  Prompt tokens: {result['usage'].prompt_tokens}")
                print(f"  Completion tokens: {result['usage'].completion_tokens}")
                print(f"  Estimated cost: ${total_cost:.6f}")
            else:
                print(f"❌ Error: {result['error']}")
        except Exception as e:
            print(f"❌ Error with vision query: {e}")
else:
    print(f"⚠️ Sample image not found at {sample_image_path}")
    print("To test vision capabilities, please provide a sample image.")
    print("You can download a sample image and place it in the same directory as this notebook.")
    print("Then run this cell again to test the vision capabilities.")

### 4D) Embeddings Tests

Test the embedding capabilities of Mistral AI for semantic search and similarity.

In [ ]:
import numpy as np

def get_embedding(text, model="mistral-embed"):
    """Get embeddings for a single text using Mistral's embedding model."""
    try:
        response = client.embeddings(model=model, input=[text])
        embedding = response.data[0].embedding
        usage = response.usage
        
        return {
            "success": True,
            "embedding": embedding,
            "usage": usage
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

def get_embeddings_batch(texts, model="mistral-embed"):
    """Get embeddings for multiple texts in a batch."""
    try:
        response = client.embeddings(model=model, input=texts)
        embeddings = [data.embedding for data in response.data]
        usage = response.usage
        
        return {
            "success": True,
            "embeddings": embeddings,
            "usage": usage
        }
    except Exception as e:
        return {"success": False, "error": str(e)}

def cosine_similarity(a, b):
    """Compute cosine similarity between two vectors."""
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

# Test embedding capabilities
print("🔍 Testing embedding capabilities")
print("-" * 50)

# Sample texts for embedding comparison
test_texts = [
    "Machine learning is a subset of artificial intelligence.",
    "AI and machine learning are transforming technology.",
    "The weather today is sunny and warm.",
    "Python is a popular programming language for data science.",
    "Data science involves analyzing and interpreting complex data."
]

# Get embeddings for all texts
try:
    result = get_embeddings_batch(test_texts)
    
    if result["success"]:
        embeddings = result["embeddings"]
        usage = result["usage"]
        
        print(f"✅ Generated embeddings for {len(embeddings)} texts")
        print(f"   Embedding dimension: {len(embeddings[0])}")
        print(f"   Total tokens used: {usage.total_tokens}")
        
        # Calculate cost
        input_cost, output_cost, total_cost = estimate_cost("mistral-embed", usage, df_pricing)
        print(f"   Estimated cost: ${total_cost:.6f}")
        
        # Calculate and display similarities
        print("\n📊 Similarity Matrix:")
        print("" + "-" * 80)
        
        similarity_matrix = []
        for i, text_a in enumerate(test_texts):
            row = []
            for j, text_b in enumerate(test_texts):
                similarity = cosine_similarity(embeddings[i], embeddings[j])
                row.append(similarity)
            similarity_matrix.append(row)
        
        # Create a DataFrame for better visualization
        df_similarity = pd.DataFrame(
            similarity_matrix,
            index=[f"Text {i+1}" for i in range(len(test_texts))],
            columns=[f"Text {i+1}" for i in range(len(test_texts))]
        )
        
        print(df_similarity.round(3))
        
        # Find most similar pairs (excluding self-similarity)
        print("\n🔗 Most similar text pairs:")
        max_similarity = 0
        most_similar_pair = None
        
        for i in range(len(test_texts)):
            for j in range(i+1, len(test_texts)):
                similarity = similarity_matrix[i][j]
                if similarity > max_similarity:
                    max_similarity = similarity
                    most_similar_pair = (i, j)
        
        if most_similar_pair:
            i, j = most_similar_pair
            print(f"   Text {i+1} & Text {j+1}: {max_similarity:.3f}")
            print(f"   '{test_texts[i]}'")
            print(f"   '{test_texts[j]}'")
        
        print("\n📝 Text Reference:")
        for i, text in enumerate(test_texts):
            print(f"   Text {i+1}: {text}")
            
    else:
        print(f"❌ Error: {result['error']}")
        
except Exception as e:
    print(f"❌ Error with embeddings: {e}")

## 5) Summary & Cost Analysis

Compare results across all tested models and analyze costs.

In [ ]:
# This cell would summarize all the results from the previous tests
# You can expand this section to create comprehensive comparisons

print("📋 Test Summary")
print("=" * 50)

if 'results' in locals() and results:
    print("\n💰 Cost Comparison for Text Models:")
    for model, data in results.items():
        print(f"  {model}: ${data['cost']:.6f}")
    
    # Find the most cost-effective model
    cheapest_model = min(results.items(), key=lambda x: x[1]['cost'])
    most_expensive_model = max(results.items(), key=lambda x: x[1]['cost'])
    
    print(f"\n🏆 Most cost-effective: {cheapest_model[0]} (${cheapest_model[1]['cost']:.6f})")
    print(f"💎 Most expensive: {most_expensive_model[0]} (${most_expensive_model[1]['cost']:.6f})")
    
    cost_ratio = most_expensive_model[1]['cost'] / cheapest_model[1]['cost']
    print(f"📊 Cost ratio (most expensive vs cheapest): {cost_ratio:.1f}x")
else:
    print("⚠️ No test results available. Run the text model tests above first.")

print("\n✅ Testing complete! Review the results above to choose the best model for your use case.")
print("\n💡 Remember to:")
print("   - Verify current pricing on https://mistral.ai/pricing/")
print("   - Test with your specific use cases")
print("   - Consider both cost and quality for your requirements")